# Exploration des données

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats 
import numpy
import seaborn as sns

### Import et présentation des données

In [19]:
path = './data'
x_data = pd.read_csv(path+'/Xtrain_hgcGIrA.csv', sep=',') # features
y_data = pd.read_csv(path+'/Ytrain_yL5OjS4.csv', sep=',', usecols=[1]) # occupancy rate

data = pd.merge(
    x_data,
    y_data,
    how="inner",
    left_index=True,
    right_index=True,
    copy=True,
    indicator=False,
    validate=None,
) # merged dataframe

data

,date,train,way,station,hour,composition,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,p0q0
0,2019-01-07,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.201,0.138,0.091,0.216
1,2019-01-08,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.204,0.152,0.106,0.216
2,2019-01-10,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.213,0.153,0.111,0.227
3,2019-01-11,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.213,0.152,0.108,0.229
4,2019-01-14,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.210,0.147,0.096,0.225
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31114,2019-05-13,9,0,BE,08:00:00,2,0.152,0.18860,0.157000,0.080,0.100,NaN,0.111
31115,2019-05-14,9,0,BE,08:00:00,2,0.153,0.18040,0.191000,0.089,0.121,NaN,0.143
31116,2019-05-15,9,0,BE,08:00:00,2,0.166,0.14900,0.168000,0.099,0.129,NaN,0.139
31117,2019-03-21,9,0,BE,08:00:00,2,0.182,0.19300,0.162000,0.074,0.101,NaN,0.117


Notre dataset d'entraînement comporte 31118 échantillons. Pour chaque ligne, on dispose des 12 features suivantes :
1. La date du jour où le trajet a lieu
2. Le numéro du train considéré de 1 à 55
3. Le sens de parcours - 0 pour dire vers Paris / 1 pour dire vers depuis Paris
4. L'identifiant de la gare où s'arrête le train
5. L'heure à laquelle le train s'arrête à la gare
6. Le nombre de rames du train (1 ou 2)
7-9. p1q0, p2q0, p3q0 : respectivement le taux d'occupation du dernier train, avant-dernier et avant-avant-dernier train au même arrêt
10-12. p0q1, p0q2, p0q3 : respectivement le taux d'occupation du même train au dernier arrêt, avant-dernier et avant-avant-dernier arrêt

On dispose aussi d'une colonne cible p0q0 : le taux d'occupation du train courant à l'arrêt courant.

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31118 entries, 1 to 31118
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         31118 non-null  object 
 1   train        31118 non-null  int64  
 2   way          31118 non-null  int64  
 3   station      31118 non-null  object 
 4   hour         27914 non-null  object 
 5   composition  31118 non-null  int64  
 6   p1q0         29067 non-null  float64
 7   p2q0         26974 non-null  float64
 8   p3q0         24935 non-null  float64
 9   p0q1         27916 non-null  float64
 10  p0q2         24719 non-null  float64
 11  p0q3         21526 non-null  float64
 12  p0q0         31118 non-null  float64
dtypes: float64(7), int64(3), object(3)
memory usage: 3.3+ MB


Afin de pouvoir mieux étudier notre dataset, nous allons rectifier certains typages fait par pandas qui ne sont pas appropriés : en effet, train et way, même si ce sont bien des entiers, doivent être considérés comme des variables catégorielles.

In [22]:
data['train'] = data['train'].astype('category')
data['way'] = data['way'].astype('category')

### Étude des données

#### Gestion des NaN

In [25]:
pd.isna(data).sum()

date              0
train             0
way               0
station           0
hour           3204
composition       0
p1q0           2052
p2q0           4145
p3q0           6184
p0q1           3202
p0q2           6399
p0q3           9592
p0q0              0
dtype: int64

In [32]:
data.loc[data['hour'].isna()] 

,date,train,way,station,hour,composition,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,p0q0
406,2019-01-07,1,0,AX,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,0.045
407,2019-01-08,1,0,AX,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,0.044
408,2019-01-10,1,0,AX,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,0.061
409,2019-01-11,1,0,AX,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,0.042
410,2019-01-14,1,0,AX,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,0.043
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30938,2019-05-13,9,0,AX,NaN,2,0.0840,0.110822,0.0970,NaN,NaN,NaN,0.080
30939,2019-05-14,9,0,AX,NaN,2,0.0940,0.100000,0.1190,NaN,NaN,NaN,0.089
30940,2019-05-15,9,0,AX,NaN,2,0.0910,0.085000,0.1060,NaN,NaN,NaN,0.099
30941,2019-03-21,9,0,AX,NaN,2,0.1070,0.107000,0.0950,NaN,NaN,NaN,0.074


Une des spécificités de notre dataset est le nombre très important de valeurs manquantes. Les colonnes concernées sont hour, p1q0, p2q0, p3q0, p0q1, p0q2, et p0q3.
La SNCF explique cela par le fait que les valeurs sont inexistantes à cause de la structure de leur grille horaire.

Plusieurs algorithmes de Machine Learning n'autorise pas des datasets avec des valeurs manquantes. Il faut donc que nous trouvions un moyen de contourner ce problème.

- Si on décide de supprimer toutes les lignes avec des valeurs NaN :

In [34]:
data_no_na = data.dropna()
data_no_na.shape

(17666, 13)

In [42]:
other_data_no_na = data.drop(labels=['p0q3', "p3q0"], axis = 1)
other_data_no_na = other_data_no_na.dropna()
other_data_no_na.shape

(21513, 11)

Si on choisit de supprimer toutes les lignes avec des valeurs NaN, on perd 43,23% des données de notre dataset. 17 666 reste un nombre important d'échantillons mais il faudrait mieux trouver un autre moyen car les lignes avec des données manquantes ont aussi des données non manquantes utiles.
On peut aussi choisir de supprimer des colonnes, par exemple p0q3 et p3q0, qui sont les données les plus "éloignées" et qui ont un nombre important de NaN. Avec cela, on conserve 21 513 lignes ce qui correspond à 215 130 données contre 211 992 données pour la solution précédente.

Il faudra cependant adapter la situation à l'algorithme choisi.

- Si on décide de ne pas supprimer les lignes avec des NaN :

Supprimer les valeurs NaN de notre dataset conduit à beaucoup de pertes, celles-ci étant nombreuses. Une meilleure option serait de faire avec ces valeurs NaN, en adaptant les algorithmes utilisés par exemple : certains acceptent des datasets avec des NaN. Cela réduit cependant notre champ d'action.    
Une autre possibilité est de compléter nos valeurs manquantes par régression linéaire, dans le cas où on dispose d'autres variables très corrélées.

Source : https://larevueia.fr/4-methodes-pour-gerer-les-donnees-manquantes-en-machine-learning-avec-pandas/

#### Statistiques chiffrés

In [14]:
data.describe()

,composition,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,p0q0
count,31118.000000,29067.000000,26974.000000,24935.000000,27916.000000,24719.000000,21526.000000,31118.000000
mean,1.999647,0.234769,0.251392,0.316867,0.207201,0.222171,0.209851,0.248748
std,0.018798,0.166153,0.143506,0.150974,0.134711,0.147603,0.144859,0.157366
min,1.000000,0.000000,0.000000,0.000000,0.004000,0.004000,0.004000,0.000000
25%,2.000000,0.094000,0.136000,0.196000,0.099000,0.107000,0.100000,0.121000
50%,2.000000,0.192000,0.230000,0.312000,0.181000,0.188000,0.172000,0.220000
75%,2.000000,0.363000,0.354000,0.420000,0.294000,0.322000,0.292000,0.360000
max,2.000000,0.934000,0.937000,0.937000,0.969000,0.974000,0.974000,0.974000


Le tableau ci-dessus nous apprend notamment que :
- La majorité des trains qui circulent sont des trains à deux rames
- En moyenne, le taux d'occupation des trains tourne autour de 0,2/0,3. Des taux d'occupation de 0,9 sont très rares (75 pourcent des valeurs sont inférieures ou égales à 0,4 au maximum pour p3q0).

In [15]:
data.median(numeric_only=True)

composition    2.000
p1q0           0.192
p2q0           0.230
p3q0           0.312
p0q1           0.181
p0q2           0.188
p0q3           0.172
p0q0           0.220
dtype: float64

Les médianes confirment nos conclusions précédentes.

#### Plots